In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ШК], [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Общий вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

In [4]:
stock

,ШК,Код склада,Код товара,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,...,Остаток,"Цена Розн., за шт",Чистый вес,Цена изделия на бирке,Скидка на изделии,Запрет скидки,Цена Закупки за 1 шт. (упр сс с НДС за единицу),Дата закупки,Код объекта,8
0,2078606623780,5979.0,ТОВ1303104,41-46060-11,41-46060-11_ЦБ,1728938,ЦБ,ЦБ БРАСЛЕТЫ,7621_ЦБ,585,...,1.0,87278.0,5.779,87278.0,6.0,Нет,17718.86,2023-10-08,4-02-053,4
1,2078606473635,5979.0,ТОВ1303246,40-56040-11,40-56040-11_ЦБ,1859625,ЦБ,ЦБ ЦЕПИ,7216_ЦБ,585,...,1.0,92261.0,6.110,92261.0,6.0,Нет,19140.83,2023-10-06,4-02-053,4
2,2078606228851,5979.0,ТОВ1304125,"НБ12-012 D0,60","НБ12-012 D0,60_ЦБ",1813496,ЦБ,ЦБ БРАСЛЕТЫ,7765_ЦБ,585,...,1.0,46782.0,2.256,46782.0,26.0,Нет,10181.86,2023-09-15,4-02-053,4
3,2078607493557,5979.0,ТОВ1304158,20-02-0001-12759,20-02-0001-12759_ИФ,1397238,ИФ,ИФ КОЛЬЦА,ИФ_1014,585,...,1.0,38412.0,1.569,37636.0,22.0,Нет,5503.49,2023-11-22,4-02-053,4
4,2078610564346,5979.0,ТОВ1304169,50-02-0001-12920,50-02-0001-12920,1806585,ИФ,ИФ ПОДВЕС ДЕКОР,ИФ_1092,585,...,1.0,29898.0,1.442,33300.0,22.0,Нет,5250.57,2024-05-04,4-02-053,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172451,2078609427266,5836.0,ТОВ1313609,20-33-1000-13115,1398903 1 5 1#БР._ДК,1847403,ДК,ДК КОЛЬЦА,ДК_2016,585,...,1.0,60920.0,1.076,29967.0,62.0,Нет,4853.32,2024-03-11,2-14-006,2
3172452,2078609307155,5836.0,ТОВ1315061,50-02-0101-80753#ТОП,50-02-0101-80753#ТОП_ПДК,1869474,ПДК,ПДК ПОДВЕСКИ,ДК_2580,585,...,1.0,26385.0,0.507,26385.0,62.0,Нет,2029.58,2024-03-05,2-14-006,2
3172453,2078588534024,4803.0,ТОВ1068635,ССК 566#ГРАН,ССК 566#ГРАН_ПДК,4845233,ПДК,ПДК СЕРЬГИ,ДК_2545,585,...,1.0,71400.0,2.780,75694.0,25.0,Нет,8410.42,None,6-02-045,6
3172454,2078605927145,4803.0,ТОВ0880918,13.15_35,13.15_35_БК,3764076,БК,БК СЕРЬГИ,БК_1561,585,...,1.0,31136.0,2.241,8736.0,5.0,Нет,6872.68,2023-08-24,6-02-045,6


In [12]:
stock = stock.groupby(by = 'ID Сайта', as_index=False)[['Остаток', 'Чистый вес', 'Общий вес']].sum()

In [15]:
stock = stock[stock['ID Сайта'] != '']

In [17]:
stock.to_excel('общий+чистый вес.xlsx', index=False)

In [13]:
stock = stock

In [76]:
df = stock

In [77]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [78]:
df.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)', 'Дата закупки'], inplace=True)
data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'], inplace=True)
df = df.merge(data, how='outer', on='ID Сайта')

In [79]:
mean_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].mean()

In [80]:
mean_weight.columns = ['ID Сайта', 'Средний вес']

In [81]:
df = df.merge(mean_weight, how='outer', on='ID Сайта')

In [82]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 9/9 [11:03<00:00, 73.76s/it]


In [83]:
objects_with_sales = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\codes.xlsx')['Код объекта'].astype(str).unique().tolist()

sale = sale.query('КБК in @objects_with_sales')
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-04-01', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-30', dayfirst=True, format='mixed'))]

In [144]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [85]:
stock.drop_duplicates(inplace = True)

In [86]:
compare_tov_id = stock[['Код товара', 'ID Сайта']]
compare_tov_id.columns = ['Тов', 'ID Сайта']

In [87]:
compare_tov_id.drop_duplicates(subset=['Тов'], inplace=True)

In [88]:
sale = sale.merge(compare_tov_id, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]
sale.dropna(inplace=True)
sale = sale.groupby(by = 'ID Сайта', as_index=False)[['Количество', 'Общая сумма']].sum()
sale.columns = ['ID Сайта', 'Количество продаж 01.04-30.06', 'Сумма продаж 01.04-30.06']

In [89]:
result = stock.drop_duplicates(subset=['ID Сайта'])
result = result.merge(sale, how='outer', on='ID Сайта')

In [90]:
ids = result['ID Сайта'].dropna().tolist()

In [91]:
images['article'] = images['article'].astype(str)
images = images.query('article in @ids')

In [92]:
images = images[['article', 'published', 'Ссылка на изображение']]
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']
result = result.merge(images, how='outer', on='ID Сайта')

In [93]:
result = result.drop_duplicates(subset=['ID Сайта'])
result = result[result['ID Сайта'] != '']

In [94]:
result = result.drop(columns=['Остаток'])

In [95]:
df = stock
df = df[df['ID Сайта'] != '']

In [124]:
stock = stock.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
result = result.merge(stock, how='left', on='ID Сайта')

In [ ]:
df['Чистый вес'] = df['Чистый вес'].astype(float)
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.rename(columns={'median': 'Чистый вес медиана', 'max': 'Чистый вес макс', 'min': 'Чистый вес мин'}, inplace=True)
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [ ]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена Розн., за шт'].agg({'min', 'median', 'max'})
price_birka.rename(columns={'median': 'Цена изделия на бирке медиана', 'max': 'Цена изделия на бирке макс', 'min': 'Цена изделия на бирке мин'}, inplace=True)
result = result.merge(price_birka, how='left', on='ID Сайта')
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) 

In [ ]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.rename(columns={'max': 'Базовая скидка макс', 'min': 'Базовая скидка мин'}, inplace=True)
result = result.merge(base_discount, how='left', on='ID Сайта')

In [130]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии'] / 100) * 0.8 * 0.6
)

In [168]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.rename(columns={'median': 'Цена со скидками медиана', 'max': 'Цена со скидками макс', 'min': 'Цена со скидками мин'}, inplace=True)
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [165]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.rename(columns={'median': 'Себестоимость медиана', 'max': 'Себестоимость макс', 'min': 'Себестоимость мин'}, inplace=True)
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) 
result = result.merge(selfprice, how='left', on='ID Сайта')

In [166]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.rename(columns={'median': 'Наценка медиана', 'max': 'Наценка макс', 'min': 'Наценка мин'}, inplace=True)
result = result.merge(markup, how='left', on='ID Сайта')

In [145]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)
zapret = zapret['Артикул сайта'].tolist()

result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [146]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.28.xlsx')

In [147]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [148]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1','discount_value_promotions_2', 'discount_value_promotions_3','discount_value_promotions_4']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [149]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [150]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]

In [151]:
ecom = ecom.query('article in @ids')
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom']
result = result.merge(ecom, how='left', on='ID Сайта')

In [152]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [153]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [156]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.rename(columns={'median': 'Цена за грамм медиана', 'max': 'Цена за грамм макс', 'min': 'Цена за грамм мин'}, inplace=True)
result = result.merge(price_per_gramm, how='left', on='ID Сайта')
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) 

In [157]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [173]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

yes = yes.merge(no, how='outer', on='ID Сайта')

result = result.merge(yes, how='left', on='ID Сайта')

In [159]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [160]:
result = result.merge(data, how='left', on='ID Сайта')

In [171]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [174]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [175]:
result = result.drop(columns = ['Код склада', 'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки', 'Цена Розн., за шт', 'Код товара'])

KeyError: "['Код склада', 'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Запрет скидки', 'Цена Розн., за шт', 'Код товара'] not found in axis"

In [ ]:
result = result.drop(columns = ['ШК'])

In [ ]:
result.columns

Index(['Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
       'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
       'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Количество продаж 01.04-30.06', 'Сумма продаж 01.04-30.06',
       'Опубликован', 'Ссылка на изображение', 'Остаток', 'Чистый вес мин',
       'Чистый вес медиана', 'Чистый вес макс', 'Чистый вес размах, %',
       'Цена изделия на бирке мин', 'Цена изделия на бирке медиана',
       'Цена изделия на бирке макс', 'Цена изделия на бирке размах, %',
       'Базовая скидка мин', 'Базовая скидка макс', 'Цена со скидками мин',
       'Цена со скидками медиана', 'Цена со скидками макс',
       'Цена со скидками размах, %', 'Себестоимость мин',
       'Себестоимость медиана', 'Себестоимость макс',
       'Себестоимость размах, %', 'Наценка мин', 'Наценка медиана',
       'Наценка макс', 'Запре

In [ ]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж 01.04-30.06'].sum()

result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'][i] = result['Остаток'][i] / tov[t] 
    
    
result['Доля % относительно продаж ТГ 01.04-30.06'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ 01.04-30.06'][i] = result['Количество продаж 01.04-30.06'][i] / tov_sales[t] 

In [54]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)

In [ ]:
date_today = dt.datetime.today().strftime('%Y-%m-%d')

In [ ]:
result.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\Исследование цен в разрезе ID {date_today}.xlsx', index=False)
# for tn in result['Товарное направление'].unique().tolist():
#     result[result['Товарное направление'] == tn].\
#         to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\Исследование цен для {tn} в разрезе ID {date_today}.xlsx', index=False)